In [5]:
import numpy as np
import pandas as pd

admissions = pd.read_csv('binary.csv')

# Make dummy variables for rank
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data = data.drop('rank', axis=1)

# Standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std
    
# Split off random 10% of the data for testing
np.random.seed(42)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.loc[sample], data.drop(sample)

# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

In [10]:
features.head()

,gre,gpa,rank_1,rank_2,rank_3,rank_4
209,-0.066657,0.289305,0,1,0,0
280,0.625884,1.445476,0,1,0,0
33,1.837832,1.603135,0,0,1,0
210,1.318426,-0.131120,0,0,0,1
93,-0.066657,-1.208461,0,1,0,0


In [11]:
targets.head()

209    0
280    0
33     1
210    0
93     0
Name: admit, dtype: int64

In [12]:
features_test.head()

,gre,gpa,rank_1,rank_2,rank_3,rank_4
20,-0.759199,-0.577822,0,0,1,0
21,0.625884,0.630901,0,1,0,0
48,-1.278605,-2.390908,0,0,0,1
50,0.452749,1.235263,0,0,1,0
54,0.625884,-0.131120,0,0,1,0


In [13]:
targets_test.head()

20    0
21    1
48    0
50    0
54    0
Name: admit, dtype: int64

In [14]:
def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))

# TODO: We haven't provided the sigmoid_prime function like we did in
#       the previous lesson to encourage you to come up with a more
#       efficient solution. If you need a hint, check out the comments
#       in solution.py from the previous lecture.error * output * (1 - output)

# Use to same seed to make debugging easier
np.random.seed(42)

n_records, n_features = features.shape
n_records, n_features

(360, 6)

In [16]:
last_loss = None

# Initialize weights
weights = np.random.normal(scale=1 / n_features**.5, size=n_features)
weights

array([ 0.64471093,  0.31330392, -0.19166212,  0.22149921, -0.18918948,
       -0.19013338])

In [17]:
# Neural Network hyperparameters
epochs = 1000
learnrate = 0.5

In [20]:
for e in range(epochs):
    del_w = np.zeros(weights.shape)
    for x, y in zip(features.values, targets):
        # Loop through all records, x is the input, y is the target

        # Note: We haven't included the h variable from the previous
        #       lesson. You can add it if you want, or you can calculate
        #       the h together with the output

        # TODO: Calculate the output
        output = sigmoid(np.dot(x, weights))

        # TODO: Calculate the error
        error = y - output

        # TODO: Calculate the error term
        error_term = error * output * (1 - output)

        # TODO: Calculate the change in weights for this sample
        #       and add it to the total weight change
        del_w += error_term * x

    # TODO: Update weights using the learning rate and the average change in weights
    weights += learnrate * del_w / n_records

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        out = sigmoid(np.dot(features, weights))
        loss = np.mean((out - targets) ** 2)
        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss


# Calculate accuracy on test data
tes_out = sigmoid(np.dot(features_test, weights))
predictions = tes_out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

Train loss:  0.24447931892257405
Train loss:  0.20656724720081765
Train loss:  0.2003430060976241
Train loss:  0.19847553939101575
Train loss:  0.19773771462427334
Train loss:  0.1973948802748109
Train loss:  0.19721774549237547
Train loss:  0.19711921039849487
Train loss:  0.1970614022820021
Train loss:  0.19702613782333592
Prediction accuracy: 0.725
